<a href="https://colab.research.google.com/github/muoyo/chicago-ridesharing/blob/master/notebooks/rideshare_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
!pip install sodapy

In [0]:
import numpy as np
import pandas as pd
import seaborn as sns
from sodapy import Socrata
import statsmodels.api as sm
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from sklearn.linear_model import LinearRegression
%matplotlib inline


In [100]:
#!/usr/bin/env python

# make sure to install these packages before running:
# pip install pandas
# !pip install sodapy

import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
# client = Socrata("data.cityofchicago.org", None)

# Example authenticated client (needed for non-public datasets):
client = Socrata('data.cityofchicago.org',
                 'Tk6RhuGAFvF9P4ehsysybj3IW',
                 username="mokome@gmail.com",
                 password="Ch1cago!!")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
client.timeout = 10000
results = client.get("m6dm-c72p", limit=2000000, select='trip_id, trip_start_timestamp, trip_end_timestamp, trip_seconds, trip_miles, pickup_community_area, dropoff_community_area, fare, tip, additional_charges, trip_total' )

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)
results_df

,trip_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area,dropoff_community_area,fare,tip,additional_charges,trip_total
0,dd0cc1463ef161e24b5cfca43d21373135e63867,2019-01-16T16:15:00.000,2019-01-16T16:30:00.000,1346,8.21715513808,7,77,15,0,2.55,17.55
1,dd0cc1b0d94700b898009174fa407127565d41ea,2018-11-01T06:00:00.000,2018-11-01T07:00:00.000,3113,20.4166496394989,45,23,17.5,0,2.5,20
2,dd0cc1d54c5ab4f104fc20ce02a8534b686e1abb,2019-01-24T19:45:00.000,2019-01-24T19:45:00.000,469,1.5670652572791,31,31,5,0,2.55,7.55
3,dd0cc3d1ed0d5a9b4ca1b29b2e50a92fe3366dc1,2018-12-21T18:15:00.000,2018-12-21T18:30:00.000,880,4.1697883216,69,39,10,0,0,10
4,dd0cc46f48999e973c01cd45468f265e844666c8,2019-03-05T14:45:00.000,2019-03-05T15:00:00.000,1506,7.32213988795855,28,29,10,0,2.55,12.55
...,...,...,...,...,...,...,...,...,...,...,...
1999995,1143f37b3053e2f9d4f37f613cdd5825c26dda78,2019-08-12T01:45:00.000,2019-08-12T02:00:00.000,1015,6.722369,5,8,12.5,3,2.55,18.05
1999996,1143f38297b979a3b88696b2244bd02341d2a500,2019-07-19T09:15:00.000,2019-07-19T09:30:00.000,1306,7.388153,32,6,15,5,2.55,22.55
1999997,1143f48e90eb89cccbff886fc4147e3c5a27fa35,2019-09-03T16:15:00.000,2019-09-03T17:00:00.000,2786,14.704924,77,76,27.5,0,7.55,35.05
1999998,1143f5069330b99ec01ae6e2da8e24836719f852,2019-09-29T18:15:00.000,2019-09-29T18:45:00.000,1753,23.44713608976,76,NaN,27.5,0,9.14,36.64


In [111]:
columns_to_use = ['trip_id', 'trip_start_timestamp', 'trip_end_timestamp', 'trip_seconds',
       'trip_miles', 'pickup_community_area', 'fare', 'tip',
       'additional_charges', 'trip_total' ]

columns_to_drop = [ col for col in results_df.columns if col not in columns_to_use ]
df = results_df.drop(columns=columns_to_drop)

df['trip_start_timestamp'] = pd.to_datetime(df['trip_start_timestamp'])
df['trip_end_timestamp'] = pd.to_datetime(df['trip_end_timestamp'])
df['trip_seconds'] = df['trip_seconds'].fillna('0')
df['trip_seconds'] = df['trip_seconds'].astype('int64')

for col in ['trip_miles', 'fare', 'tip', 'additional_charges', 'trip_total']:
    df[col] = df[col].astype(float) 

df['start_weekday'] = df['trip_start_timestamp'].apply(lambda d: d.weekday())
df['start_hour'] = df['trip_start_timestamp'].apply(lambda d: d.hour)
df['start_time_block'] = df['start_hour'] // 3

df['start_date_plus_hour'] = df['trip_start_timestamp'].apply(lambda d: datetime(d.year, d.month, d.day, d.hour))

weather_df = pd.read_csv('https://github.com/muoyo/chicago-ridesharing/raw/master/data/chicago_weather.csv')
weather_df['hour'] = weather_df['hour'].apply(lambda x: '{:02d}'.format(x))
weather_df['start_date_plus_hour'] = pd.to_datetime(weather_df['date'] + ' ' + weather_df['hour'] + ':00:00')
weather_df = weather_df.rename(columns={'icon': 'precip'})

# def set_precip(precip):
#     if precip not in ['rain', 'snow']: 
#         precip = 'clear'

#     return precip

# weather_df['precip'] = weather_df['precip'].apply(set_precip)
precip_df = weather_df[['start_date_plus_hour', 'precip', 'apparentTemperature']]

df = df.merge(precip_df, how='left', on='start_date_plus_hour')
df.head()

,trip_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area,fare,tip,additional_charges,trip_total,start_weekday,start_hour,start_time_block,start_date_plus_hour,precip,apparentTemperature
0,dd0cc1463ef161e24b5cfca43d21373135e63867,2019-01-16 16:15:00,2019-01-16 16:30:00,1346,8.217155,7,15.0,0.0,2.55,17.55,2,16,5,2019-01-16 16:00:00,clear,21.94
1,dd0cc1b0d94700b898009174fa407127565d41ea,2018-11-01 06:00:00,2018-11-01 07:00:00,3113,20.416650,45,17.5,0.0,2.50,20.00,3,6,2,2018-11-01 06:00:00,clear,50.02
2,dd0cc1d54c5ab4f104fc20ce02a8534b686e1abb,2019-01-24 19:45:00,2019-01-24 19:45:00,469,1.567065,31,5.0,0.0,2.55,7.55,3,19,6,2019-01-24 19:00:00,clear,-1.47
3,dd0cc3d1ed0d5a9b4ca1b29b2e50a92fe3366dc1,2018-12-21 18:15:00,2018-12-21 18:30:00,880,4.169788,69,10.0,0.0,0.00,10.00,4,18,6,2018-12-21 18:00:00,clear,21.29
4,dd0cc46f48999e973c01cd45468f265e844666c8,2019-03-05 14:45:00,2019-03-05 15:00:00,1506,7.322140,28,10.0,0.0,2.55,12.55,1,14,4,2019-03-05 14:00:00,clear,6.71


In [112]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000000 entries, 0 to 1999999
Data columns (total 16 columns):
trip_id                  object
trip_start_timestamp     datetime64[ns]
trip_end_timestamp       datetime64[ns]
trip_seconds             int64
trip_miles               float64
pickup_community_area    object
fare                     float64
tip                      float64
additional_charges       float64
trip_total               float64
start_weekday            int64
start_hour               int64
start_time_block         int64
start_date_plus_hour     datetime64[ns]
precip                   object
apparentTemperature      float64
dtypes: datetime64[ns](3), float64(6), int64(4), object(3)
memory usage: 259.4+ MB


In [0]:
# def preprocess(X, y, cont_vars, cat_vars, random_state=False):
#     '''Takes in features and target and implements all preprocessing steps for categorical and continuous features returning 
#     train and test DataFrames with targets'''
    
#     # Train-test split (75-25), set seed to 10
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=random_state)
    
#     # Remove "object"-type features from X
#     # cont_features = [col for col in X.columns if X[col].dtype in [np.float64, np.int64]]
#     cont_features = [col for col in cont_vars ]

#     X_train_cont = X_train.loc[:, cont_features]
#     X_test_cont = X_test.loc[:, cont_features]

#     X_train_cont = X_train_cont.replace([np.inf, -np.inf], X_train_cont.median)
#     X_test_cont = X_test_cont.replace([np.inf, -np.inf], X_test_cont.median)

#     # Impute missing values with median using SimpleImputer
#     impute = SimpleImputer(strategy='median')

#     X_train_imputed = impute.fit_transform(X_train_cont)
#     X_test_imputed = impute.transform(X_test_cont)



#     # Scale the train and test data
#     ss = StandardScaler()

#     X_train_imputed_scaled = ss.fit_transform(X_train_imputed)
#     X_test_imputed_scaled = ss.transform(X_test_imputed)

#     # Create X_cat which contains only the categorical variables
#     features_cat = [col for col in cat_vars]
#     X_train_cat = X_train.loc[:, features_cat]
#     X_test_cat = X_test.loc[:, features_cat]

#     # Fill nans with a value indicating that that it is missing
#     X_train_cat.fillna(value='missing', inplace=True)
#     X_test_cat.fillna(value='missing', inplace=True)

#     # OneHotEncode Categorical variables
#     ohe = OneHotEncoder(handle_unknown='ignore')

#     X_train_ohe = ohe.fit_transform(X_train_cat)
#     X_test_ohe = ohe.transform(X_test_cat)

#     columns = ohe.get_feature_names(input_features=X_train_cat.columns)
#     cat_train_df = pd.DataFrame(X_train_ohe.todense(), columns=columns, index=X_train_cat.index)
#     cat_test_df = pd.DataFrame(X_test_ohe.todense(), columns=columns, index=X_test_cat.index)
    
#     # Combine categorical and continuous features into the final dataframe
#     X_train_all = pd.concat([pd.DataFrame(X_train_imputed_scaled), cat_train_df], axis=1)
#     X_test_all = pd.concat([pd.DataFrame(X_test_imputed_scaled), cat_test_df], axis=1)
    
#     return X_train_all, X_test_all, y_train, y_test

In [174]:
columns_to_use = ['start_weekday', 'start_hour', 'pickup_community_area']

columns_to_drop = [ col for col in df.columns if col not in columns_to_use ]
X = df.drop(columns=columns_to_drop)
# X['apparentTemperature']=X['apparentTemperature'].fillna(0)
X['pickup_community_area']=X['pickup_community_area'].fillna('0')



enc = OneHotEncoder()

y = df['trip_total']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Transform training and test sets
X_train_enc = enc.fit_transform(X_train, y_train)
X_test_enc = enc.transform(X_test)

# Convert these columns into a DataFrame 
columns = enc.get_feature_names(input_features=X_train.columns)
X_train = pd.DataFrame(X_train_enc.todense(), columns=columns, index=X_train.index)
X_test = pd.DataFrame(X_test_enc.todense(), columns=columns, index=X_test.index)

# Combine categorical and continuous features into the final dataframe
# X_train_all = pd.concat([pd.DataFrame(X_train_imputed_scaled), X_train], axis=1)
# X_test_all = pd.concat([pd.DataFrame(X_test_imputed_scaled), X_test], axis=1)

X_train = sm.add_constant(X_train)
X_test = sm.add_constant(X_test)
model = sm.OLS(y_train,X_train).fit()
model.summary()

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             trip_total   R-squared:                       0.279
Model:                            OLS   Adj. R-squared:                  0.279
Method:                 Least Squares   F-statistic:                     5473.
Date:                Wed, 22 Jan 2020   Prob (F-statistic):               0.00
Time:                        16:33:48   Log-Likelihood:            -5.5610e+06
No. Observations:             1500000   AIC:                         1.112e+07
Df Residuals:                 1499893   BIC:                         1.112e+07
Df Model:                         106                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                       12.3538      0.017    746.586      0.000      12.321      12.386
pickup_community_area_0     13.4293      0.037    365.585      0.000      13.357      13.501
pickup_community_area_1      0.0915      0.081      1.133      0.257      -0.067       0.250
pickup_community_area_10     0.9284      0.145      6.410      0.000       0.645       1.212
pickup_community_area_11     0.2057      0.154      1.338      0.181      -0.096       0.507
pickup_community_area_12     2.3135      0.220     10.514      0.000       1.882       2.745
pickup_community_area_13    -0.3590      0.175     -2.056      0.040      -0.701      -0.017
pickup_community_area_14    -0.5111      0.106     -4.823      0.000      -0.719      -0.303
pickup_community_area_15     0.0625      0.103      0.605      0.545      -0.140       0.265
pickup_community_area_16    -0.3342      0.084     -3.962      0.000      -0.500      -0.169
pickup_community_area_17     0.0455      0.157      0.290      0.772      -0.262       0.352
pickup_community_area_18    -0.4159      0.250     -1.667      0.096      -0.905       0.073
pickup_community_area_19    -1.4020      0.097    -14.435      0.000      -1.592      -1.212
pickup_community_area_2     -0.8220      0.093     -8.792      0.000      -1.005      -0.639
pickup_community_area_20    -1.5514      0.160     -9.675      0.000      -1.866      -1.237
pickup_community_area_21    -1.3088      0.080    -16.349      0.000      -1.466      -1.152
pickup_community_area_22    -1.4408      0.046    -31.087      0.000      -1.532      -1.350
pickup_community_area_23    -1.7710      0.094    -18.922      0.000      -1.954      -1.588
pickup_community_area_24    -1.9241      0.037    -51.571      0.000      -1.997      -1.851
pickup_community_area_25    -1.3268      0.082    -16.166      0.000      -1.488      -1.166
pickup_community_area_26    -2.3785      0.170    -13.971      0.000      -2.712      -2.045
pickup_community_area_27    -1.6882      0.130    -13.009      0.000      -1.943      -1.434
pickup_community_area_28    -1.2955      0.033    -38.815      0.000      -1.361      -1.230
pickup_community_area_29    -0.8432      0.102     -8.232      0.000      -1.044      -0.642
pickup_community_area_3     -0.6758      0.059    -11.524      0.000      -0.791      -0.561
pickup_community_area_30    -1.1738      0.105    -11.152      0.000      -1.380      -0.967
pickup_community_area_31    -1.3413      0.076    -17.696      0.000      -1.490      -1.193
pickup_community_area_32     0.4530      0.033     13.900      0.000       0.389       0.517
pickup_community_area_33     1.1450      0.058     19.870      0.000       1.032       1.258
pickup_community_area_34    -0.1440      0.111     -1.293      0.196      -0.362       0.074
pickup_communi

In [0]:
# columns_to_use = ['apparentTemperature', 'pickup_community_area']

# columns_to_drop = [ col for col in df.columns if col not in columns_to_use ]
# X = df.drop(columns=columns_to_drop)
# X['apparentTemperature']=X['apparentTemperature'].fillna(0)
# X['pickup_community_area']=X['pickup_community_area'].fillna('0')



# enc = OneHotEncoder()

# y = df['trip_total']

# X_train, X_test, y_train, y_test = preprocess(X, y, ['apparentTemperature'], ['pickup_community_area'])

# # # Split data into training and test sets
# # X_train, X_test, y_train, y_test = train_test_split(X_cat, y, test_size=0.25)

# # # Transform training and test sets
# # X_train_enc = enc.fit_transform(X_train, y_train)
# # X_test_enc = enc.transform(X_test)

# # # Convert these columns into a DataFrame 
# # columns = enc.get_feature_names(input_features=X_train.columns)
# # X_train_df = pd.DataFrame(X_train_enc.todense(), columns=columns, index=X_train.index)
# # X_test_df = pd.DataFrame(X_test_enc.todense(), columns=columns, index=X_test.index)

# X_train = sm.add_constant(X_train)
# X_test = sm.add_constant(X_test)
# model = sm.OLS(y_train,X_train).fit()
# model.summary()

In [157]:
X.loc[X['pickup_community_area'].isna()]

,pickup_community_area,apparentTemperature


In [153]:
# Fit the model
linreg = LinearRegression()
linreg.fit(X_train_df, y_train)

# Print R2 and MSE for training and test sets
print('Training r^2:', linreg.score(X_train_df, y_train))
print('Test r^2:', linreg.score(X_test_df, y_test))
print('Training MSE:', mean_squared_error(y_train, linreg.predict(X_train_df)))
print('Test MSE:', mean_squared_error(y_test, linreg.predict(X_test_df)))


Training r^2: 6.345178282074482e-05
Test r^2: -5.901503661998042e-05
Training MSE: 134.82059802867286
Test MSE: 133.82620002347434


In [110]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10223 entries, 0 to 10222
Data columns (total 22 columns):
date                    10223 non-null object
hour                    10223 non-null object
time                    10223 non-null int64
summary                 10223 non-null object
precip                  10223 non-null object
precipIntensity         10223 non-null float64
precipProbability       10223 non-null float64
precipType              7033 non-null object
temperature             10223 non-null float64
apparentTemperature     10223 non-null float64
dewPoint                10223 non-null float64
humidity                10223 non-null float64
pressure                10223 non-null float64
windSpeed               10223 non-null float64
windGust                10223 non-null float64
windBearing             10223 non-null int64
cloudCover              10223 non-null float64
uvIndex                 10223 non-null int64
visibility              10223 non-null float64
ozone     